## Evaluating Proprietary Models

## Environmental Setup

In [ ]:
!pip install openai
!pip install zhipuai

If you run this file locally, make sure to save api keys in `.env` file. 

In [1]:
import base64
import sys
import os
import csv 
from openai import OpenAI
from zhipuai import ZhipuAI
import pandas as pd

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    from google.colab import userdata
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    GLM_MODEL_API_KEY = userdata.get('GLM_MODEL_API_KEY')
else:
    from dotenv import load_dotenv

    load_dotenv()
    OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
    GLM_MODEL_API_KEY = os.getenv('GLM_MODEL_API_KEY')

## Batch Evaluation

Define functions for batch evaluation.

In [2]:
# Function to create a request for each image
def process_images_in_batch(image_folder, prompt, model, verbose=True):
    # get all image files in the folder
    image_files = [f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
    
    # create client based on the model    
    if model == "gpt-4o":
        client = OpenAI(api_key=OPENAI_API_KEY)
    elif model == "glm-4v-plus":
        client = ZhipuAI(api_key=GLM_MODEL_API_KEY)
    else:
        raise("The model name must be either 'gpt-4o' or 'glm-4v-plus'.")
        
    responses = []
    
    # process each image
    for image_file in image_files:        
        image_path = os.path.join(image_folder, image_file)
        
        ## temporary code to skip certain images
        index = int(image_file.split('_')[-1].split('.')[0])
        if index not in [11, 15, 20, 29, 30, 31]:
            continue
        
        # read the image file and convert it to base64
        with open(image_path, "rb") as f:
            base64_image = base64.b64encode(f.read()).decode('utf-8')
            
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[
                    {
                        "role": "user",
                        "content": [
                            {
                                "type": "text",
                                "text": prompt,                                
                            },
                            {
                                "type": "image_url",
                                "image_url": {
                                    "url": f"data:image/jpeg;base64,{base64_image}"
                                },
                            },
                        ],
                    }
                ],
                max_tokens=120,
                temperature=0.3,
                top_p=0.6
            )
            
            responses.append({"image_file": image_file,"response": response.choices[0].message.content})

            if verbose:
                print(f"Filename: {image_file}| Response: {response.choices[0].message.content}")
            
        except Exception as e:
            print(f"Error processing image {image_path}: {e}")
            continue
    
    return responses

def save_responses(responses, image_folder, csv_file_path, first_write=True, verbose=False):          
    ### The file needs to exist if it's not the first write 
    if not first_write and not os.path.exists(csv_file_path):
        print(f"The provided csv file does not exist. Saving in {csv_file_path}")
    
    ### determining original_country and synthesized_race automatically from the folder name
    possible_countries = ['Korea', 'UK', 'Myanmar', 'Azerbaijan', 'US']
    possible_synthesized_races = ['African', 'EastAsian', 'SouthAsian', 'Caucasian']

    original_country = [country for country in possible_countries if country in image_folder][0]

    if 'original' in image_folder:
        synthesized_race = original_country
    else:
        synthesized_race = [race for race in possible_synthesized_races if race in image_folder][0]
    
    ### saving into the csv file
    with open(csv_file_path, mode='a', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        
        # Write the header if it's the first write
        if first_write:
            csv_writer.writerow(['original_country', 'synthesized_race', 'image_file_name', 'response'])

        # Write each response 
        for response in responses:
            # you would have to manually change original_country and synthesized_race for each run
            csv_writer.writerow([original_country, synthesized_race, response['image_file'], response['response']])
            
            if verbose: print(f"Filename: {response['image_file']} | Response: {response['response']}")
            
        print(f"Data saved to: {csv_file_path}")
        
    ### sort the rows in the .csv file by file index
    df = pd.read_csv(csv_file_path)
    df["index"] = df["image_file_name"].apply(lambda file_name: int(file_name.split("_")[-1].split(".")[0]))
    df_sorted = df.sort_values(by=["synthesized_race", "index"], ascending=[True, True])
    df_sorted = df_sorted.drop(columns=["index"])
    df_sorted.to_csv(csv_file_path, index=False)

Evaluate all images inside a given directory and save the responses in a given `.csv` file.
- Set `first_write=True` if writing responses into a new file and needing the **header row** like `'original_country', 'synthesized_race', 'image_file_name', 'response'`.
- The function will automatically figure out `original_country` and `synthesized_race` from `image_folder`, so ensure the folder path name is meaningful.

In [ ]:
model = "gpt-4o"
# model = "glm-4v-plus"

# prompt = "Which country is the celebration/cultural activity/festival in the photo the most associated with? Which visual cues did you use to determine it?"
# prompt = "Which country is the food in the photo the most associated with? Which food is it?"
prompt = "Which country is the clothing in the photo the most associated with? Name the clothes and which visual cues did you use to determine it?"

image_category = 'US_Clothes'
csv_file_path = f"../responses/gpt4o/{image_category}_Results.csv"

### For original images
image_folder = f"../images/{image_category}/original_images/"
responses = process_images_in_batch(image_folder, prompt, model)
save_responses(responses, image_folder, csv_file_path, first_write=True)

### For synthesized images
races = ['African', 'EastAsian', 'SouthAsian', 'Caucasian']
for _, race in enumerate(races):
    image_folder = f"../images/{image_category}/synthesized_images/{race}"
    responses = process_images_in_batch(image_folder, prompt, model)
    save_responses(responses, image_folder, csv_file_path, first_write=False)


In [3]:
# model = "gpt-4o"
model = "glm-4v-plus"

# prompt = "Which country is the celebration/cultural activity/festival in the photo the most associated with? Which visual cues did you use to determine it?"
# prompt = "Which country is the food in the photo the most associated with? Which food is it?"
prompt = "Which country is the clothing in the photo the most associated with? Name the clothes and which visual cues did you use to determine it?"
image_category = 'US_Clothes'
csv_file_path = f"../responses/glm4v/{image_category}_Results.csv"

## For original images
image_folder = f"../images/{image_category}/original_images/"
responses = process_images_in_batch(image_folder, prompt, model)
save_responses(responses, image_folder, csv_file_path, first_write=False)

Filename: US_Clothes_11.png| Response: The clothing in the photo is most associated with the United States. The visual cues include cowboy hats, button-up shirts with suspenders, and jeans, which are iconic elements of traditional Western wear commonly linked to American cowboy culture.
Filename: US_Clothes_29.png| Response: The clothing in the photo is most associated with Peru. The traditional outfits, featuring colorful embroidery and distinctive red hats, are characteristic of Peruvian indigenous attire, often seen in regions like the Andes. The visual cues include the vibrant patterns, the style of the hats, and the layered, embroidered garments typical of Peruvian cultural dress.
Filename: US_Clothes_30.png| Response: The clothing in the photo is most associated with Native American culture, specifically from the United States. The visual cues include the feathered headdress, the beadwork, and the turquoise jewelry, which are traditional elements commonly linked to various Native